In [3]:
import pandas as pd
import time 
import geocoder
import numpy as np

In [4]:
df=pd.read_csv('E:\Project\CLI_BB\input_file_clients.csv')

In [7]:
df.sample(5)

,Email,First Name,Last Name,Residential Address Street,Residential Address Locality,Residential Address State,Residential Address Postcode,Postal Address Street,Postal Address Locality,Postal Address State,Postal Address Postcode
5597,marcia@gmail.com,Phoenix,Wilson,84 Sam Terrace,ADDINGTON,VIC,3352,6598 Leo Street,ANNANGROVE,NSW,2156.0
7661,denita_goldner@yahoo.com,Leah,Beahan,6086 Lowe Mall,ASHBOURNE,VIC,3442,74 Emma Crescent,ARTARMON,NSW,2064.0
6900,maurice.hoppe@hotmail.com,Jasper,Collier,969 Muller Drive,ALMONDS,VIC,3727,2947 Jett Plaza,ALEXANDRA HEADLAND,QLD,4572.0
8580,glynis@gmail.com,Lucas,Goldner,12 Ryan Junction,ANNERLEY,QLD,4103,62 Larson Circle,AMBERGATE,WA,6280.0
2144,alvaro_mayert@hotmail.com,Amelia,Adams,20 Morrison Junction,ARAWATA,VIC,3951,202 Bartoletti Drive,ABINGTON,QLD,4660.0


In [ ]:
df.dropna(inplace=True)

df['Residential Address Postcode'] = df['Residential Address Postcode'].astype(str)
df['Postal Address Postcode'] = df['Postal Address Postcode'].astype(str)
df['Postal Address Postcode'] = df['Postal Address Postcode'].str.replace('.0', '')

In [4]:
df['Residential Address Full']=df['Residential Address Locality'] + ', ' + df['Residential Address State'] + ', ' + df['Residential Address Postcode']

df['Postal Address Full']=df['Postal Address Locality'] + ', ' + df['Postal Address State'] + ', ' + df['Postal Address Postcode']

In [5]:
batch_size = 1000
batches = [df['Residential Address Full'][i:i+batch_size] for i in range(0, len(df), batch_size)]

In [6]:
def geocode_address(address):
    num_retries = 0
    retry=3
    while num_retries < retry:
        location = geocoder.osm(address)
        try:
            if location.ok:
                return (location.lat, location.lng)
            else:
                print(f"Failed to geocode address: {address}")
                return np.nan
        except Exception as e:
            num_retries += 1
            time.sleep(0.5)
            continue 
    print(f'failed after retry for this add, {address}')
    return np.nan


In [ ]:
results = []
for i, batch in enumerate(batches):
    batch_results = [geocode_address(address) for address in batch]
    results.extend(batch_results)


df['Residential co-ordinates'] = pd.Series(results)
import warnings 
warnings.filterwarnings('ignore')

In [9]:
df.to_csv('updated_file_clients.csv', index=False)